**Simple example for Asg1 with handcrafted feature**

An example diagram for instance search.
Please read the slide "Information for Asg1" first.

In every bounding box file (the txt files), each line records the coordinates of one bounding box in format: x of top-left point, y of top-left point, width, height.

The bounding box information is only provided for the query images.

Notes: it's possible that there are more than one instances in a query image (see the annotation(s) in the corresponding txt file), you should find the most similar image rank list in the gallery by considering all the instances.

In [13]:
import numpy as np 
import cv2 
import pandas as pd
import glob
import time 

 Download the dataset from this linke [onedrive link](https://portland-my.sharepoint.com/:u:/g/personal/srwang3-c_my_cityu_edu_hk/EZ0BIZatMIJMoiCG4-uy6okBSlXLJD7TUyIDG1lbKUJ0eA?e=lbKz4a).
And unzip the downloaded file into some path. 
> In this tutorial, the path is `/Users/txsing/datasets_4186`, please replace to your own accordingly.

Initialize the necessary parameters, including paths, feature extractors.

In [14]:
# download_path='/Users/jethrotsoi/Downloads/AssignmentOne/CS4186_dataset' # change to your own download path
download_path='//Users/jethrotsoi/git/CS4186/AssignmentOne/CS4186_dataset'
path_query=download_path+'/query_img_4186'
path_query_txt=download_path+'/query_img_box_4186'

# path_query_txt is the directory to the bounding box information of the instance(s) for the query images
path_gallery=download_path+'/gallery_4186'

name_query=glob.glob(path_query+'/*.jpg')
num_query=len(name_query)
name_gallery=glob.glob(path_gallery+'/*.jpg')
num_gallery=len(name_gallery)
sift = cv2.ORB_create()
record_all=np.zeros((num_query,len(name_gallery)))

query_imgs_no = [x.split('/')[-1][:-4] for x in glob.glob(path_query+'/*.jpg')]
gallery_imgs_no = [x.split('/')[-1][:-4] for x in glob.glob(path_gallery+'/*.jpg')]

In [15]:
print(query_imgs_no)

['2714', '776', '3557', '2461', '1709', '316', '2176', '1656', '4716', '3906', '35', '1258', '4929', '4445', '27', '2032', '3502', '2040', '4354', '3833']


In [16]:
class SIFT_Result:
    def __init__(self, kp, des):
        self.per_query_kp = kp
        self.per_query_des = des

Process

Feature extraction -> calculate distance -> compare

In [17]:
# # the iteration loop for query 
# # iteration times is 2 here, only for the demonstration]
        
# query_sift_results = {}
# for i, query_img_no in enumerate(query_imgs_no[0:2]):
#     time_s = time.time()
#     dist_record=[]
#     per_query_name=path_query+'/'+str(query_img_no)+'.jpg'
#     per_query=cv2.imread(per_query_name)
#     # feature extraction for per query
#     # the bounding box information is not considered

#     # Add bounding box information to the query image
#     per_query_txt_name = path_query_txt+'/'+str(query_img_no)+'.txt'
#     # Load the bounding box information
#     with open(per_query_txt_name, 'r') as f:
#         bbox = [int(x) for x in f.read().split()]

#     # Crop the image using bounding box
#     x, y, w, h = bbox
#     crop_img = per_query[y:y+h, x:x+w]

#     # Apply SIFT to cropped image
#     # sift = cv2.xfeatures2d.SIFT_create()
#     per_query_kp, per_query_des = sift.detectAndCompute(crop_img, None)

#     query_sift_results[query_img_no] = SIFT_Result(per_query_kp, per_query_des)

#     # # quite naive, just an example
#     # per_query_kp, per_query_des = sift.detectAndCompute(per_query,None)


#     # the iteration loop for gallery
#     for j, gallery_img_no in enumerate(gallery_imgs_no):
#         per_gallery_name = path_gallery+'/'+str(gallery_img_no)+'.jpg'
#         per_gallery=cv2.imread(per_gallery_name)
#         # feature extraction for per gallery
#         per_gallery_kp, per_gallery_des = sift.detectAndCompute(per_gallery,None)
#         # use part of the features to make the calculation feasible
#         # quite naive, just an example
#         min_kp_num=np.amin([len(per_query_kp),len(per_gallery_kp)])
#         query_part=per_query_des[0:min_kp_num,:]
#         gallery_part=per_gallery_des[0:min_kp_num,:]
#         # distance calculation in feature domain (similarity)
#         dist_record.append(np.sum((np.double(query_part)-np.double(gallery_part))**2)/np.prod(np.shape(query_part)))
#     # find the indexes with descending similarity order
#     ascend_index=sorted(range(len(dist_record)), key=lambda k: dist_record[k])
#     # update the results for one query 
#     record_all[i,:]=ascend_index
#     time_e = time.time()
#     print('retrieval time for query {} is {}s'.format(query_img_no, time_e-time_s))

In [18]:
# import pickle
# import os
# save_gallery_sift_results_path = '/Users/jethrotsoi/git/CS4186/AssignmentOne/SIFT/data/SIFT/gallery_sift_results.pkl'
gallery_sift_results = {}
# if os.path.exists(save_gallery_sift_results_path):
#     with open(save_gallery_sift_results_path, 'rb') as f:
#         gallery_sift_results = pickle.load(f)

In [19]:
# Calculate SIFT results for all gallery images
for gallery_img_no in gallery_imgs_no:
    if gallery_img_no in gallery_sift_results:
        continue
    per_gallery_name = path_gallery+'/'+str(gallery_img_no)+'.jpg'
    per_gallery = cv2.imread(per_gallery_name)
    per_gallery_kp, per_gallery_des = sift.detectAndCompute(per_gallery, None)
    gallery_sift_results[gallery_img_no] = SIFT_Result(per_gallery_kp, per_gallery_des)
    
# # Save the dictionary into a pickle file
# with open(save_gallery_sift_results_path, 'wb') as f:
#     pickle.dump(gallery_sift_results, f)

TypeError: cannot pickle 'cv2.KeyPoint' object

In [ ]:
# Calculate SIFT results for query images and compare with gallery images
query_sift_results = {}
for i, query_img_no in enumerate(query_imgs_no[0:2]):
    time_s = time.time()
    dist_record = []
    per_query_name = path_query+'/'+str(query_img_no)+'.jpg'
    per_query = cv2.imread(per_query_name)

        # Add bounding box information to the query image
    per_query_txt_name = path_query_txt+'/'+str(query_img_no)+'.txt'
    # Load the bounding box information
    with open(per_query_txt_name, 'r') as f:
        bbox = [int(x) for x in f.read().split()]

    # Crop the image using bounding box
    x, y, w, h = bbox
    crop_img = per_query[y:y+h, x:x+w]

    # Apply SIFT to cropped image
    # sift = cv2.xfeatures2d.SIFT_create()
    per_query_kp, per_query_des = sift.detectAndCompute(crop_img, None)

    query_sift_results[query_img_no] = SIFT_Result(per_query_kp, per_query_des)

    # Compare with each gallery image
    for gallery_img_no in gallery_imgs_no:
        sift_result = gallery_sift_results[gallery_img_no]
        per_gallery_kp = sift_result.per_query_kp  # assuming 'kp' is the attribute name for keypoints
        per_gallery_des = sift_result.per_query_des  # assuming 'des' is the attribute name for descriptors
        min_kp_num = np.amin([len(per_query_kp), len(per_gallery_kp)])
        query_part = per_query_des[0:min_kp_num, :]
        gallery_part = per_gallery_des[0:min_kp_num, :]
        dist_record.append(np.sum((np.double(query_part)-np.double(gallery_part))**2)/np.prod(np.shape(query_part)))

    # find the indexes with descending similarity order
    ascend_index=sorted(range(len(dist_record)), key=lambda k: dist_record[k])
    # update the results for one query 
    record_all[i,:]=ascend_index
    time_e = time.time()
    print('retrieval time for query {} is {}s'.format(query_img_no, time_e-time_s))

FileNotFoundError: [Errno 2] No such file or directory: '/Users/jethrotsoi/git/CS4186/AssignmentOne/SIFT/data/SIFT/gallery_sift_results.pkl'

In [ ]:
query_idx = 0
print(f'For query image No. {query_imgs_no[query_idx]}, the top ranked similar image No. is {gallery_imgs_no[int(record_all[query_idx][-1])]}.')

For query image No. 2714, the top ranked similar image No. is 82.


Output 

Save the rank list file.

In [ ]:
# write the output file following the example
f=open(r'./rank_list.txt','w')
for i in range(num_query):
    f.write('Q'+str(i+1)+': ')
    for j in range(len(name_gallery)):
        f.write(str(np.int32(record_all[i,j]))+' ')
    f.write('\n')
f.close()